In [5]:
!pip install emoji

In [2]:
import pandas as pd
import numpy as np
import re
import emoji
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [129]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML/11.project2/gender_detection/data/train_data.csv')
data

,gender,age,fullname,username,biography,follower_count,following_count,is_business,is_verified,is_private
0,man,2,Farshid,mr_gh_farshid,دردا ک در این بادیه بسیار دویدیم...\nGlory man...,1604.0,1407.0,0.0,0.0,0.0
1,woman,2,zahr@72,zahra.roozbahani72,"خواهی که زکوچ در امان برگردی\nباید که به جان ,...",67.0,501.0,0.0,0.0,0.0
2,woman,2,ms farahnaz♥,___lady.farahnazi.__,"Having you, is all I wish for \nداشتنت، تمامِ...",0.0,0.0,0.0,0.0,0.0
3,woman,1,Lena.mommy farzan,mommy.lena3361,دردونه من لنا کوچولو,0.0,0.0,0.0,0.0,0.0
4,woman,2,Narsis Asadollahi,_l.aurora.l_,I am an animation student\n🎧🎼🎨⚓️🤍 \n@general.m...,200.0,328.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7995,woman,2,Ŋεgɨŋ,negiiin_bahrmandi,﷽\nAllah IS Enough FoR Me?\n♥️?,0.0,0.0,0.0,0.0,0.0
7996,man,3,h🗯abdi🗯offìcial,h.abdi.official,حقوقی,0.0,0.0,1.0,0.0,0.0
7997,woman,2,⚜رویا احمدی⚜,roya.ahmadi.k,مهندس صنایع👩‍🔧🏭 Industrial engineer\nمعمار👩‍💻👷...,1260.0,1167.0,0.0,0.0,0.0
7998,man,3,لرستان &خرم اباد,erfanpouersif,khoramabad,0.0,0.0,0.0,0.0,0.0


In [130]:
# splitting the dataset
train, test = train_test_split(data, test_size = 0.2)

In [131]:
# checking for missing data
print(train.isna().sum())
print(test.isna().sum())

gender             0
age                0
fullname           0
username           0
biography          0
follower_count     0
following_count    0
is_business        3
is_verified        0
is_private         0
dtype: int64
gender             0
age                0
fullname           0
username           0
biography          0
follower_count     0
following_count    0
is_business        0
is_verified        0
is_private         0
dtype: int64


In [132]:
# filling missing values in 'is_business' column with the most frequent value (mode)
train['is_business'] = train['is_business'].fillna(train['is_business'].mode()[0])
train

,gender,age,fullname,username,biography,follower_count,following_count,is_business,is_verified,is_private
571,woman,4,Sabah Mayahi Manesh,sabah_mayahi_manesh,artist _ painter,435.0,1401.0,0.0,0.0,0.0
5517,woman,2,soraya.safarzade,sorayya.safarzade9496,تا در دلِ ♡من قرار کردی!\nدل را زِ تو بی قرار ...,402.0,847.0,0.0,0.0,0.0
527,man,3,‌PERSIAN—BOY,mr_matin3577,🇮🇷ENGINEER OF MEHR ABAD INTERNATIONAL AIRPORT🛫...,0.0,0.0,0.0,0.0,0.0
4266,man,4,hossin_noroznezhad,hossin_noroznezhad,##سینما#تاتر#کمدی##طنز,1466.0,530.0,0.0,0.0,0.0
4900,man,4,حسين كاجي,housein.kaji,تنها صفحه شخصي بنده در اينستاگرام(تازه واردم)\...,4383.0,96.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
598,man,4,Navid Madannezhad,navid.mnzh,is learning,0.0,0.0,0.0,0.0,0.0
5081,man,3,amin,amin.m900,Turk❤Urmia👑\n.\n⚽🚜🚜🚜⚽\n.\n👇\n😁اسوووون نينرررر😏...,0.0,0.0,0.0,0.0,0.0
7939,man,3,گالری حکاکی ۱۲۱,hakaki121,🌹🌹بسم الله الرحمن الرحیم🌹🌹 \n لبیک ...,4303.0,7448.0,0.0,0.0,0.0
6477,woman,3,Raana_rh_,_.raana.__,#❤💎\nInterior Architecture 🎨📏🖌🏡🏛✏📚\n🔘 NOVACOLO...,193.0,2707.0,0.0,0.0,0.0


In [133]:
# mapping 'man' to 0 and 'woman' to 1 in the 'gender' column
train['gender'] = train['gender'].map({'man': 0, 'woman': 1})
test['gender'] = test['gender'].map({'man': 0, 'woman': 1})

In [134]:
# function to clean and preprocess text by removing unwanted characters, digits, and extra spaces, then converting to lowercase
def preprocess_text(text):
    text = re.sub(r'[^\w\s,.!?]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()

In [135]:
# applying the text preprocessing function to 'fullname', 'username', and 'biography' columns in the train data
train['fullname'] = train['fullname'].apply(preprocess_text)
train['username'] = train['username'].apply(preprocess_text)
train['biography'] = train['biography'].apply(preprocess_text)
train

,gender,age,fullname,username,biography,follower_count,following_count,is_business,is_verified,is_private
571,1,4,sabah mayahi manesh,sabah_mayahi_manesh,artist _ painter,435.0,1401.0,0.0,0.0,0.0
5517,1,2,soraya.safarzade,sorayya.safarzade,تا در دل من قرار کردی! دل را ز تو بی قرار دیدم...,402.0,847.0,0.0,0.0,0.0
527,0,3,persianboy,mr_matin,engineer of mehr abad international airport のε...,0.0,0.0,0.0,0.0,0.0
4266,0,4,hossin_noroznezhad,hossin_noroznezhad,سینماتاترکمدیطنز,1466.0,530.0,0.0,0.0,0.0
4900,0,4,حسين كاجي,housein.kaji,تنها صفحه شخصي بنده در اينستاگرامتازه واردم ha...,4383.0,96.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
598,0,4,navid madannezhad,navid.mnzh,is learning,0.0,0.0,0.0,0.0,0.0
5081,0,3,amin,amin.m,turkurmia . . اسوووون نينرررر گنع من!!!يوخ,0.0,0.0,0.0,0.0,0.0
7939,0,3,گالری حکاکی,hakaki,بسم الله الرحمن الرحیم لبیک یا زینبس هذا من فض...,4303.0,7448.0,0.0,0.0,0.0
6477,1,3,raana_rh_,_.raana.__,interior architecture novacolor italia مجری پو...,193.0,2707.0,0.0,0.0,0.0


In [136]:
# applying the text preprocessing function to 'fullname', 'username', and 'biography' columns in the test data
test['fullname'] = test['fullname'].apply(preprocess_text)
test['username'] = test['username'].apply(preprocess_text)
test['biography'] = test['biography'].apply(preprocess_text)

In [137]:
# function to count the number of emojis in a given text
def count_emojis(text):
    return len([char for char in text if char in emoji.EMOJI_DATA])

In [138]:
# applying the 'count_emojis' function to the 'biography' column to create 'emoji_count' in the train data
train['emoji_count'] = train['biography'].apply(count_emojis)
test['emoji_count'] = test['biography'].apply(count_emojis)

In [139]:
# calculating the word count in 'biography' and create 'bio_length' column for both train and test data
train['bio_length'] = train['biography'].apply(lambda x: len(x.split()))
test['bio_length'] = test['biography'].apply(lambda x: len(x.split()))

In [140]:
train

,gender,age,fullname,username,biography,follower_count,following_count,is_business,is_verified,is_private,emoji_count,bio_length
571,1,4,sabah mayahi manesh,sabah_mayahi_manesh,artist _ painter,435.0,1401.0,0.0,0.0,0.0,0,3
5517,1,2,soraya.safarzade,sorayya.safarzade,تا در دل من قرار کردی! دل را ز تو بی قرار دیدم...,402.0,847.0,0.0,0.0,0.0,0,22
527,0,3,persianboy,mr_matin,engineer of mehr abad international airport のε...,0.0,0.0,0.0,0.0,0.0,0,7
4266,0,4,hossin_noroznezhad,hossin_noroznezhad,سینماتاترکمدیطنز,1466.0,530.0,0.0,0.0,0.0,0,1
4900,0,4,حسين كاجي,housein.kaji,تنها صفحه شخصي بنده در اينستاگرامتازه واردم ha...,4383.0,96.0,0.0,0.0,0.0,0,13
...,...,...,...,...,...,...,...,...,...,...,...,...
598,0,4,navid madannezhad,navid.mnzh,is learning,0.0,0.0,0.0,0.0,0.0,0,2
5081,0,3,amin,amin.m,turkurmia . . اسوووون نينرررر گنع من!!!يوخ,0.0,0.0,0.0,0.0,0.0,0,7
7939,0,3,گالری حکاکی,hakaki,بسم الله الرحمن الرحیم لبیک یا زینبس هذا من فض...,4303.0,7448.0,0.0,0.0,0.0,0,12
6477,1,3,raana_rh_,_.raana.__,interior architecture novacolor italia مجری پو...,193.0,2707.0,0.0,0.0,0.0,0,10


In [141]:
# applying TF-IDF vectorization to the combined text columns ('fullname', 'username', 'biography') for both train and test data
tfidf = TfidfVectorizer()
X_train_text = tfidf.fit_transform(train['fullname'] + ' ' + train['username'] + ' ' + train['biography'])
X_test_text = tfidf.transform(test['fullname'] + ' ' + test['username'] + ' ' + test['biography'])

In [142]:
# training an XGBoost classifier model using the TF-IDF features from the text data to predict 'gender'
txt_model = XGBClassifier()
txt_model.fit(X_train_text, train['gender'])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [143]:
# Predict 'gender' on the train and test sets
train['gender_by_txt'] = txt_model.predict(X_train_text)
test['gender_by_txt'] = txt_model.predict(X_test_text)

In [144]:
train['gender_by_txt']

,gender_by_txt
571,1
5517,1
527,1
4266,1
4900,0
...,...
598,0
5081,0
7939,0
6477,1


In [145]:
# selecting relevant columns for training and test data, including features and predictions
train_data = train[['age', 'follower_count', 'is_business', 'is_verified', 'is_private', 'emoji_count', 'bio_length', 'gender_by_txt']]
test_data = test[['age', 'follower_count', 'is_business', 'is_verified', 'is_private', 'emoji_count', 'bio_length', 'gender_by_txt']]

In [146]:
train_data

,age,follower_count,is_business,is_verified,is_private,emoji_count,bio_length,gender_by_txt
571,4,435.0,0.0,0.0,0.0,0,3,1
5517,2,402.0,0.0,0.0,0.0,0,22,1
527,3,0.0,0.0,0.0,0.0,0,7,1
4266,4,1466.0,0.0,0.0,0.0,0,1,1
4900,4,4383.0,0.0,0.0,0.0,0,13,0
...,...,...,...,...,...,...,...,...
598,4,0.0,0.0,0.0,0.0,0,2,0
5081,3,0.0,0.0,0.0,0.0,0,7,0
7939,3,4303.0,0.0,0.0,0.0,0,12,0
6477,3,193.0,0.0,0.0,0.0,0,10,1


In [147]:
# training the final XGBoost classifier model using the selected features from train_data to predict 'gender'
final_model = XGBClassifier()
final_model.fit(train_data, train['gender'])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [148]:
# predicting on the train and test sets
train_preds = final_model.predict(train_data)
test_preds = final_model.predict(test_data)

In [151]:
# model evaluation
score_1 = f1_score(train['gender'], train_preds)
print(f"f1 score on Training Data: {score_1}")

score_2 = f1_score(test['gender'], test_preds)
print(f"f1 score on Test Data: {score_2}")

f1 score on Training Data: 0.906872954477834
f1 score on Test Data: 0.7922987164527421
